# EDA For the Data Files
## Detect file encoding

In [12]:
import chardet

def detect_encoding(file):
    with open(file, 'rb') as file:
        result = chardet.detect(file.read())
        encoding = result['encoding']
        print(f"Detected encoding for {file} : {encoding}")

list_of_csvs=[r'Customers.csv',r'Products.csv',r'Sales.csv']

for i in list_of_csvs:
    detect_encoding(i)
              

Detected encoding for <_io.BufferedReader name='Customers.csv'> : Windows-1252
Detected encoding for <_io.BufferedReader name='Products.csv'> : ascii
Detected encoding for <_io.BufferedReader name='Sales.csv'> : ascii


## EDA 

In [13]:
#File Customers
import pandas as pd
df_customers=pd.DataFrame()
df_customers=pd.read_csv(list_of_csvs[0],encoding='windows-1252')

df_customers.head()

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965


In [16]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerKey  15266 non-null  int64         
 1   Gender       15266 non-null  object        
 2   Name         15266 non-null  object        
 3   City         15266 non-null  object        
 4   State Code   15256 non-null  object        
 5   State        15266 non-null  object        
 6   Zip Code     15266 non-null  object        
 7   Country      15266 non-null  object        
 8   Continent    15266 non-null  object        
 9   Birthday     15266 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 1.2+ MB


In [17]:
#Convert Birthday to Datetime format
df_customers['Birthday']=pd.to_datetime(df_customers['Birthday'], format='%m/%d/%Y')

In [18]:
#check for null values
df_customers.isna().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [19]:
#seeing what the null values are
df_customers[df_customers.isnull().any(axis=1)]

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
5304,729681,Female,Rossana Padovesi,Polvica,NaN,Napoli,80035,Italy,Europe,1981-04-18
5316,732289,Male,Indro Piccio,Varcaturo,NaN,Napoli,80014,Italy,Europe,1949-02-24
5372,742042,Male,Amaranto Loggia,Casaferro,NaN,Napoli,80034,Italy,Europe,1936-03-14
5377,742886,Female,Edmonda Capon,Terzigno,NaN,Napoli,80040,Italy,Europe,1963-08-06
5378,743343,Female,Ambra Sagese,Pomigliano D'Arco,NaN,Napoli,80038,Italy,Europe,1961-01-05
5485,759705,Male,Callisto Lo Duca,Casilli,NaN,Napoli,80047,Italy,Europe,1976-08-28
5525,765589,Male,Michelino Lucchesi,Pompei Scavi,NaN,Napoli,80045,Italy,Europe,1947-11-13
5531,766410,Male,Adelmio Beneventi,Licola,NaN,Napoli,80078,Italy,Europe,1940-01-13
5631,781667,Female,Ilda Manna,Napoli,NaN,Napoli,80134,Italy,Europe,1977-05-08
5695,789177,Male,Calogero Folliero,Mariglianella,NaN,Napoli,80030,Italy,Europe,2000-03-03


In [20]:
#Replacing State Code null values for Naploli as NA as it is the state code for Napoli 
df_customers.fillna('NA',inplace=True)

#removing spaces in column names as it is better for sql
df_customers.columns = df_customers.columns.str.replace(' ', '')

In [21]:
#checking for duplicated values
df_customers.duplicated().sum()

0

In [22]:
df_customers.head(1)

,CustomerKey,Gender,Name,City,StateCode,State,ZipCode,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03


In [23]:
from datetime import datetime
def calculate_age(birthday):
    today = datetime.now()
    age = today.year - birthday.year - ((today.month, today.day) < (birthday.month, birthday.day))
    return age
    
#Creating copy as we would need original celaning data to input into sql, table wise

df_customers['Age'] = df_customers['Birthday'].apply(calculate_age)

def agerange(age):
    if age < 10:
        return 'Under 10'
    elif age < 20:
        return '10-19'
    elif age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    else:
        return '60 and above'

df_customers['AgeRange'] = df_customers['Age'].apply(agerange)


In [24]:
# can drop few columns

df_customers.drop(columns = ['StateCode','ZipCode','Continent','Birthday'], inplace = True)
df_customers.head(1)

,CustomerKey,Gender,Name,City,State,Country,Age,AgeRange
0,301,Female,Lilly Harding,WANDEARAH EAST,South Australia,Australia,85,60 and above


In [25]:
df_customers['AgeRange'].value_counts()

AgeRange
60 and above    6794
50-59           2267
30-39           2267
40-49           2263
20-29           1675
Name: count, dtype: int64

In [26]:
#File Products

df_products=pd.DataFrame()
df_products=pd.read_csv(list_of_csvs[1],encoding='ascii')

df_products.head()

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


In [27]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ProductKey      2517 non-null   int64 
 1   Product Name    2517 non-null   object
 2   Brand           2517 non-null   object
 3   Color           2517 non-null   object
 4   Unit Cost USD   2517 non-null   object
 5   Unit Price USD  2517 non-null   object
 6   SubcategoryKey  2517 non-null   int64 
 7   Subcategory     2517 non-null   object
 8   CategoryKey     2517 non-null   int64 
 9   Category        2517 non-null   object
dtypes: int64(3), object(7)
memory usage: 196.8+ KB


In [28]:
df_products.describe()

,ProductKey,SubcategoryKey,CategoryKey
count,2517.000000,2517.000000,2517.000000
mean,1259.000000,491.810091,4.878824
std,726.739637,229.887134,2.299170
min,1.000000,101.000000,1.000000
25%,630.000000,305.000000,3.000000
50%,1259.000000,406.000000,4.000000
75%,1888.000000,801.000000,8.000000
max,2517.000000,808.000000,8.000000


In [29]:
#Checking for null values
df_products.isna().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [30]:
#checking for duplicate values
df_products.duplicated().sum()

0

In [31]:
#Convert Unit Cost USD and Unit Price USD to float while removing $ and , for computational purposes
df_products['Unit Cost USD'] = df_products['Unit Cost USD'].str.replace(r'[\$,]', '', regex=True).astype(float)
df_products['Unit Price USD'] = df_products['Unit Price USD'].str.replace(r'[\$,]', '', regex=True).astype(float)

In [32]:
#Replacing spaces in column name
df_products.columns = df_products.columns.str.replace(' ', '')

In [33]:
df_products.head(1)

,ProductKey,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio


In [34]:
# Can remove unwanted columns
df_products.drop(columns= ['SubcategoryKey','CategoryKey'], inplace= True)
df_products.head(1)

,ProductKey,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,Subcategory,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,MP4&MP3,Audio


In [35]:
#File Sales

df_sales=pd.DataFrame()
df_sales=pd.read_csv(list_of_csvs[2],encoding='ascii')
df_sales.head()

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [36]:
#Replacing spaces in column name
df_sales.columns = df_sales.columns.str.replace(' ', '')

In [37]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   OrderNumber   62884 non-null  int64 
 1   LineItem      62884 non-null  int64 
 2   OrderDate     62884 non-null  object
 3   DeliveryDate  13165 non-null  object
 4   CustomerKey   62884 non-null  int64 
 5   StoreKey      62884 non-null  int64 
 6   ProductKey    62884 non-null  int64 
 7   Quantity      62884 non-null  int64 
 8   CurrencyCode  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [38]:
df_sales.describe()

,OrderNumber,LineItem,CustomerKey,StoreKey,ProductKey,Quantity
count,6.288400e+04,62884.000000,6.288400e+04,62884.000000,62884.000000,62884.000000
mean,1.430905e+06,2.164207,1.180797e+06,31.802144,1125.859344,3.144790
std,4.532963e+05,1.365170,5.859634e+05,22.978188,709.244010,2.256371
min,3.660000e+05,1.000000,3.010000e+02,0.000000,1.000000,1.000000
25%,1.121017e+06,1.000000,6.808580e+05,8.000000,437.000000,1.000000
50%,1.498016e+06,2.000000,1.261200e+06,37.000000,1358.000000,2.000000
75%,1.788010e+06,3.000000,1.686496e+06,53.000000,1650.000000,4.000000
max,2.243032e+06,7.000000,2.099937e+06,66.000000,2517.000000,10.000000


In [39]:
#Checking for null 
df_sales.isna().sum()

OrderNumber         0
LineItem            0
OrderDate           0
DeliveryDate    49719
CustomerKey         0
StoreKey            0
ProductKey          0
Quantity            0
CurrencyCode        0
dtype: int64

In [40]:
#Changin Delivery Date and Order Date to Datetime format for analysis purpose
df_sales['DeliveryDate'] = pd.to_datetime(df_sales['DeliveryDate'], errors='coerce')
df_sales['OrderDate'] = pd.to_datetime(df_sales['OrderDate'])

In [41]:
#filling null delivery date values to same day as order date as it was bought in store, doing this only for non-online orders or can ask the company team in real world case
df_sales.loc[
    df_sales['DeliveryDate'].isna() & (df_sales['StoreKey'] != 0), 
    'DeliveryDate'
] = df_sales['OrderDate']

In [42]:
# can drop irrelevant columns

In [43]:
df_sales.head(1)

,OrderNumber,LineItem,OrderDate,DeliveryDate,CustomerKey,StoreKey,ProductKey,Quantity,CurrencyCode
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD


In [44]:
df_sales = df_sales[df_sales['StoreKey'] == 0]
df_sales['StoreKey'].value_counts()

StoreKey
0    13165
Name: count, dtype: int64

In [45]:
# we chose only one store i.e online orders
# now we can remove storeKey, and currency code too and Delivery Date too

In [46]:
df_sales.drop(columns = ['DeliveryDate','CurrencyCode', 'StoreKey'],inplace=True)
df_sales.head(1)

,OrderNumber,LineItem,OrderDate,CustomerKey,ProductKey,Quantity
1,366001,1,2016-01-01,1269051,1048,2


In [65]:
# stock level add

# Create a stock_level DataFrame from ProductKey
df_products['StockLevel'] = 10  # Set default stock level to 10

df_products.head(1)

,ProductKey,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,Subcategory,Category,StockLevel
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,MP4&MP3,Audio,10


## Write to Mysql 

In [55]:
from sqlalchemy import create_engine, text
# Database connection parameters
user = 'root'
password = 'password'
host = 'localhost'  # or your MySQL host
database = 'crpm'

# Create SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

In [69]:
# Defining the SQL command to create the table Customers
create_table_query1 = """
CREATE TABLE IF NOT EXISTS Customers (
    CustomerKey INT PRIMARY KEY,
    Gender VARCHAR(10),
    Name VARCHAR(255),
    City VARCHAR(255),
    State VARCHAR(255),
    Country VARCHAR(100),
    Age INT,
    AgeRange VARCHAR(20)
)
"""

# Execute the create table command
with engine.connect() as connection:
    connection.execute(text(create_table_query1))

#Write the Customer contents into Mysql
df_customers.to_sql('Customers', con=engine, if_exists='append', index=False)


# Defining the SQL command to create the table Products
create_table_query2 = """
CREATE TABLE IF NOT EXISTS Products (
    ProductKey INT PRIMARY KEY,
    ProductName VARCHAR(255),
    Brand VARCHAR(100),
    Color VARCHAR(50),
    UnitCostUSD FLOAT,
    UnitPriceUSD FLOAT,
    Subcategory VARCHAR(255),
    Category VARCHAR(255),
    StockLevel INT,
    IsActive BOOLEAN DEFAULT TRUE
)
"""

# Execute the create table command
with engine.connect() as connection:
    connection.execute(text(create_table_query2))
    
#Write the Products contents into Mysql
df_products.to_sql('Products', con=engine, if_exists='append', index=False)


# Defining the SQL command to create the table Sales
create_table_query3 = """
CREATE TABLE IF NOT EXISTS Sales (
    OrderNumber INT,
    LineItem INT,
    OrderDate DATE,
    CustomerKey INT,
    ProductKey INT,
    Quantity INT,
    PRIMARY KEY (OrderNumber, LineItem)
)
"""

# Execute the create table command
with engine.connect() as connection:
    connection.execute(text(create_table_query3))

#Write the Sales contents into Mysql
df_sales.to_sql('Sales', con=engine, if_exists='append', index=False)

13165

In [58]:
df_customers.head(1)

,CustomerKey,Gender,Name,City,State,Country,Age,AgeRange
0,301,Female,Lilly Harding,WANDEARAH EAST,South Australia,Australia,85,60 and above


In [59]:
df_products.head(1)

,ProductKey,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,Subcategory,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,MP4&MP3,Audio


In [60]:
df_sales.head(1)

,OrderNumber,LineItem,OrderDate,CustomerKey,ProductKey,Quantity
1,366001,1,2016-01-01,1269051,1048,2
